<a href="https://colab.research.google.com/github/maciejgorczak/phishing/blob/master/phishing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Phishing
####this needs to be described in upcoming weeks.
https://archive.ics.uci.edu/ml/datasets/phishing+websites



In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#wczytanie danych wcześniej otwartych w excelu (najbardziej dogodna forma odczytu formatu .arff)
import pandas as pd
data =pd.read_excel('/content/gdrive/My Drive/phising.xlsx')
print(data)

       @attribute having_IP_Address    ...  @attribute Result  
0                                  -1  ...                   -1
1                                   1  ...                   -1
2                                   1  ...                   -1
3                                   1  ...                   -1
4                                   1  ...                    1
...                               ...  ...                  ...
11050                               1  ...                    1
11051                              -1  ...                   -1
11052                               1  ...                   -1
11053                              -1  ...                   -1
11054                              -1  ...                   -1

[11055 rows x 31 columns]


In [0]:
#wczytanie pozostałych bibliotek wykorzystywanych w projekcie
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data
from scipy.stats import pearsonr

opisanie datasetu
wczytanie danych
biblioteki


In [0]:
#opisanie statystyczne zestawu danych
print(data.describe())

#jak się mają unikalne wartości w atrybutach
print(data['@attribute age_of_domain  '].value_counts())

#nieobrobiony nagłówek danych
data.head()

       @attribute having_IP_Address    ...  @attribute Result  
count                    11055.000000  ...         11055.000000
mean                         0.313795  ...             0.113885
std                          0.949534  ...             0.993539
min                         -1.000000  ...            -1.000000
25%                         -1.000000  ...            -1.000000
50%                          1.000000  ...             1.000000
75%                          1.000000  ...             1.000000
max                          1.000000  ...             1.000000

[8 rows x 31 columns]
 1    5866
-1    5189
Name: @attribute age_of_domain  , dtype: int64


,@attribute having_IP_Address,@attribute URL_Length,@attribute Shortining_Service,@attribute having_At_Symbol,@attribute double_slash_redirecting,@attribute Prefix_Suffix,@attribute having_Sub_Domain,@attribute SSLfinal_State,@attribute Domain_registeration_length,@attribute Favicon,@attribute port,@attribute HTTPS_token,@attribute Request_URL,@attribute URL_of_Anchor,@attribute Links_in_tags,@attribute SFH,@attribute Submitting_to_email,@attribute Abnormal_URL,@attribute Redirect,@attribute on_mouseover,@attribute RightClick,@attribute popUpWidnow,@attribute Iframe,@attribute age_of_domain,@attribute DNSRecord,@attribute web_traffic,@attribute Page_Rank,@attribute Google_Index,@attribute Links_pointing_to_page,@attribute Statistical_report,@attribute Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,1,-1,1,-1,1,-1,-1,-1,0,1,1,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,1,-1,1,0,-1,-1,1,1,0,1,1,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,1,-1,1,0,-1,-1,-1,-1,0,1,1,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,1,-1,-1,0,0,-1,1,1,0,1,1,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,1,1,1,0,0,-1,1,1,0,-1,1,-1,1,-1,-1,0,-1,1,1,1,1


In [0]:
#sprawdzenie, czy dane są kompletne
print(data.isnull().any())

@attribute having_IP_Address               False
@attribute URL_Length                      False
@attribute Shortining_Service              False
@attribute having_At_Symbol                False
@attribute double_slash_redirecting        False
@attribute Prefix_Suffix                   False
@attribute having_Sub_Domain               False
@attribute SSLfinal_State                  False
@attribute Domain_registeration_length     False
@attribute Favicon                         False
@attribute port                            False
@attribute HTTPS_token                     False
@attribute Request_URL                     False
@attribute URL_of_Anchor                   False
@attribute Links_in_tags                   False
@attribute SFH                             False
@attribute Submitting_to_email             False
@attribute Abnormal_URL                    False
@attribute Redirect                        False
@attribute on_mouseover                    False
@attribute RightClic

In [0]:
#zabieg kosmetyczny - zmiana nazw kolumn
cols = []
for col in data.columns:
  col = col.split(" ")[1]
  col = col.split("_")
  if type(col)==list:
    colo = []
    for elem in col:
      if elem.isupper() == False:
        elem = elem.capitalize()
        colo.append(elem)
      else:
        colo.append(elem)
    colo = " ".join(colo) 
    cols.append(colo)

data.columns = cols
data.head()

,Having IP Address,URL Length,Shortining Service,Having At Symbol,Double Slash Redirecting,Prefix Suffix,Having Sub Domain,Sslfinal State,Domain Registeration Length,Favicon,Port,HTTPS Token,Request URL,URL Of Anchor,Links In Tags,SFH,Submitting To Email,Abnormal URL,Redirect,On Mouseover,Rightclick,Popupwidnow,Iframe,Age Of Domain,Dnsrecord,Web Traffic,Page Rank,Google Index,Links Pointing To Page,Statistical Report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,1,-1,1,-1,1,-1,-1,-1,0,1,1,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,1,-1,1,0,-1,-1,1,1,0,1,1,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,1,-1,1,0,-1,-1,-1,-1,0,1,1,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,1,-1,-1,0,0,-1,1,1,0,1,1,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,1,1,1,0,0,-1,1,1,0,-1,1,-1,1,-1,-1,0,-1,1,1,1,1


In [0]:
#sprawdzenie unikalnych wartości poszczególnych atrybutów
#tutaj można dorzucić link do pliku opisującego cały zestaw danych
for col in data.columns:
  print(data[col].value_counts())

 1    7262
-1    3793
Name: Having IP Address, dtype: int64
-1    8960
 1    1960
 0     135
Name: URL Length, dtype: int64
 1    9611
-1    1444
Name: Shortining Service, dtype: int64
 1    9400
-1    1655
Name: Having At Symbol, dtype: int64
 1    9626
-1    1429
Name: Double Slash Redirecting, dtype: int64
-1    9590
 1    1465
Name: Prefix Suffix, dtype: int64
 1    4070
 0    3622
-1    3363
Name: Having Sub Domain, dtype: int64
 1    6331
-1    3557
 0    1167
Name: Sslfinal State, dtype: int64
-1    7389
 1    3666
Name: Domain Registeration Length, dtype: int64
 1    9002
-1    2053
Name: Favicon, dtype: int64
 1    9553
-1    1502
Name: Port, dtype: int64
 1    9259
-1    1796
Name: HTTPS Token, dtype: int64
 1    6560
-1    4495
Name: Request URL, dtype: int64
 0    5337
-1    3282
 1    2436
Name: URL Of Anchor, dtype: int64
 0    4449
-1    3956
 1    2650
Name: Links In Tags, dtype: int64
-1    8440
 1    1854
 0     761
Name: SFH, dtype: int64
 1    9041
-1    2014
Name: 

In [0]:
#przygotowanie danych do policzenia korelacji między nimi
attributes = data.drop("Result", axis=1)
target = data["Result"]
attributes_cor = attributes.columns.tolist()
target_cor = target.name

In [0]:
#współczynnik korelacji Pearsona między danymi (atrybutami), a atrybutem decyzyjnym
correlations = {}
for d in attributes_cor:
    data_temp = data[[d,target_cor]]
    x1 = data_temp[d].values
    x2 = data_temp[target_cor].values
    key = d + ' vs ' + target_cor
    correlations[key] = pearsonr(x1,x2)[0]
data_correlations = pd.DataFrame(correlations, index=['Result']).T
data_correlations.loc[data_correlations['Result'].abs().sort_values(ascending=False).index]

,Result
Sslfinal State vs Result,0.714741
URL Of Anchor vs Result,0.692935
Prefix Suffix vs Result,0.348606
Web Traffic vs Result,0.346103
Having Sub Domain vs Result,0.298323
Request URL vs Result,0.253372
Links In Tags vs Result,0.248229
Domain Registeration Length vs Result,-0.225789
SFH vs Result,0.221419
Google Index vs Result,0.128950


In [0]:
#wybranie i usunięcie atrybutów nieskorelowanych z danymi
columns_to_drop = ['Domain Registeration Length', 'Shortining Service', 'Abnormal URL', 'HTTPS Token', 'Double Slash Redirecting',
                   'Redirect', 'Iframe', 'Favicon']
data = data.drop(columns_to_drop, axis=1)

In [0]:
print(data.head())

   Having IP Address  URL Length  ...  Statistical Report  Result
0                 -1           1  ...                  -1      -1
1                  1           1  ...                   1      -1
2                  1           0  ...                  -1      -1
3                  1           0  ...                   1      -1
4                  1           0  ...                   1       1

[5 rows x 23 columns]


In [0]:
#zmiana wartości z -1 i 1 do 0 i 1
data['Rezultat'] = 1
data.Rezultat[data.Result < 0] = 0
data.head()
data.drop(columns='Result', inplace=True)
data.head

<bound method NDFrame.head of        Having IP Address  URL Length  ...  Statistical Report  Rezultat
0                     -1           1  ...                  -1         0
1                      1           1  ...                   1         0
2                      1           0  ...                  -1         0
3                      1           0  ...                   1         0
4                      1           0  ...                   1         1
...                  ...         ...  ...                 ...       ...
11050                  1          -1  ...                   1         1
11051                 -1           1  ...                   1         0
11052                  1          -1  ...                   1         0
11053                 -1          -1  ...                   1         0
11054                 -1          -1  ...                  -1         0

[11055 rows x 23 columns]>

In [0]:
#dzielenie danych na zbiór treningowy i testowy
mask = np.random.rand(len(data)) < 0.75
train = data[mask]
test = data[~mask]
print(train.shape, test.shape)

(8295, 23) (2760, 23)


In [0]:
# hiperparameters = [16,32,64,128]
# batch = [20, 50, 70, 100]
# learning = [0.0005 , 0.001, 0.002]
# di = {}
# for i in hiperparameters:
#     #Parametry sieci i narzędzia ładującego dane
#     inputSize = len(train.columns) -1  #wejście naszej sieci, równe ilości atrybutów
#     hidden1Size = i #ilość perceptronów na pierwszej warstwie naszej sieci (i jej wyjście)
#     hidden2Size = i #ilość perceptronów na drugiej warstwie naszej sieci (i jej wyjście)
#     numClasses = 2 #wyjście z trzeciej swarstwy sieci, czyli z całej sieci
#     numEpoch = 100 #liczba epok, podczas któryvh nasza sieć będzie się uczyła
#     print(i)
#     bat = {}
#     for b in batch:
#         batchSize = b #ilość próbek danych wkorzystywanych do nauki modelu
#         print(b)
#         ler = {}
#         for l in learning:
#             learningRate = l #stała uczenia sieci
#             print(l)

#             #wczytywanie danych za pomocą biblioteki pytorch, pozwalajace na uczenie sieci na mniejszych partiach danych, umożliwia ich wygodne ładowanie do sieci, oraz mieszanie
#             trainLoader = torch.utils.data.DataLoader(dataset=torch.tensor(train.values), batch_size=batchSize, shuffle=True)
#             testLoader = torch.utils.data.DataLoader(dataset=torch.tensor(test.values))
#             #bierzemy testow i na nim sprawdzamy, na calym bez dzielenia na batches

#             #siec neuronowa
#             class DeepNeuralNetwork(nn.Module):
#                 def __init__(self, inputSize, hidden1Size, hidden2Size, numClasses):
#                     super(DeepNeuralNetwork, self).__init__()
#                     self.fc1 = nn.Linear(inputSize, hidden1Size)
#                     self.relu1 = nn.ReLU()
#                     self.fc2 = nn.Linear(hidden1Size, hidden2Size)
#                     self.relu2 = nn.ReLU()
#                     self.fc3 = nn.Linear(hidden2Size, numClasses)
#                     self.logsm1 = nn.LogSoftmax(dim=1)
                    
#                 def forward(self, x):
#                     out = self.fc1(x)
#                     out = self.relu1(out)
#                     out = self.fc2(out)
#                     out = self.relu2(out)
#                     out = self.fc3(out)
#                     out = self.logsm1(out)
#                     return out

#             dnn = DeepNeuralNetwork(inputSize, hidden1Size, hidden2Size, numClasses)

#             #funkcja straty i optymizer naszej sieci
#             lossFN = nn.NLLLoss()
#             optimizer = torch.optim.Adam(dnn.parameters(), lr=learningRate)

#             #trenowanie sieci neuronowej
#             for epoch in range(0, numEpoch):
#                 for i, data in enumerate(trainLoader,0):
#                     labels = Variable(data[:,-1])
#                     data = Variable(data[:,0:22].float())
#                     optimizer.zero_grad()
#                     outputs = dnn(data)
#                     loss = lossFN(outputs, labels.long())
#                     loss.backward()
#                     optimizer.step() 
                                                    
#                 #print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, numEpoch, loss.item()))

#             #sprawdzanie danych na całym zbiorze testowym
#             correct = 0
#             total = 0
#             for data in testLoader:
#                 labels = Variable(data[:,-1])
#                 data = Variable(data[:,0:22].float()) 
#                 outputs = dnn(data)
#                 _, predicted = torch.max(outputs.data, 1)  
#                 total += labels.size(0)
#                 correct += int((predicted == labels.long()).sum())
#             print('Accuracy of the network on the data: {:.4f}%'.format(100 * correct / total))
#             bas = float("{:.4f}".format(100 * correct / total))
#             ler[l] = bas
#         bat[b] = ler
#     di[i] = bat

In [0]:

#Parametry sieci i narzędzia ładującego dane
inputSize = len(train.columns) -1  #wejście naszej sieci, równe ilości atrybutów
hidden1Size = 128 #ilość perceptronów na pierwszej warstwie naszej sieci (i jej wyjście)
hidden2Size = 128 #ilość perceptronów na drugiej warstwie naszej sieci (i jej wyjście)
numClasses = 2 #wyjście z trzeciej swarstwy sieci, czyli z całej sieci
numEpoch = 100 #liczba epok, podczas któryvh nasza sieć będzie się uczyła
batchSize = 50 #ilość próbek danych wkorzystywanych do nauki mode
learningRate = 0.0005 #stała uczenia sieci


#wczytywanie danych za pomocą biblioteki pytorch, pozwalajace na uczenie sieci na mniejszych partiach danych, umożliwia ich wygodne ładowanie do sieci, oraz mieszanie
trainLoader = torch.utils.data.DataLoader(dataset=torch.tensor(train.values), batch_size=batchSize, shuffle=True)
testLoader = torch.utils.data.DataLoader(dataset=torch.tensor(test.values))
#bierzemy testow i na nim sprawdzamy, na calym bez dzielenia na batches

#siec neuronowa
class DeepNeuralNetwork(nn.Module):
  def __init__(self, inputSize, hidden1Size, hidden2Size, numClasses):
      super(DeepNeuralNetwork, self).__init__()
      self.fc1 = nn.Linear(inputSize, hidden1Size)
      self.relu1 = nn.ReLU()
      self.fc2 = nn.Linear(hidden1Size, hidden2Size)
      self.relu2 = nn.ReLU()
      self.fc3 = nn.Linear(hidden2Size, numClasses)
      self.logsm1 = nn.LogSoftmax(dim=1)
      
  def forward(self, x):
      out = self.fc1(x)
      out = self.relu1(out)
      out = self.fc2(out)
      out = self.relu2(out)
      out = self.fc3(out)
      out = self.logsm1(out)
      return out

dnn = DeepNeuralNetwork(inputSize, hidden1Size, hidden2Size, numClasses)

#funkcja straty i optymizer naszej sieci
lossFN = nn.NLLLoss()
optimizer = torch.optim.Adam(dnn.parameters(), lr=learningRate)

#trenowanie sieci neuronowej
for epoch in range(0, numEpoch):
  for i, data in enumerate(trainLoader,0):
      labels = Variable(data[:,-1])
      data = Variable(data[:,0:22].float())
      optimizer.zero_grad()
      outputs = dnn(data)
      loss = lossFN(outputs, labels.long())
      loss.backward()
      optimizer.step() 
                                      
  print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, numEpoch, loss.item()))

#sprawdzanie danych na całym zbiorze testowym
correct = 0
total = 0
for data in testLoader:
  labels = Variable(data[:,-1])
  data = Variable(data[:,0:22].float()) 
  outputs = dnn(data)
  _, predicted = torch.max(outputs.data, 1)  
  total += labels.size(0)
  correct += int((predicted == labels.long()).sum())
print('Accuracy of the network on the data: {:.4f}%'.format(100 * correct / total))


Epoch [1/100], Loss: 0.1705
Epoch [2/100], Loss: 0.2906
Epoch [3/100], Loss: 0.1768
Epoch [4/100], Loss: 0.1898
Epoch [5/100], Loss: 0.0992
Epoch [6/100], Loss: 0.0978
Epoch [7/100], Loss: 0.1976
Epoch [8/100], Loss: 0.0904
Epoch [9/100], Loss: 0.0712
Epoch [10/100], Loss: 0.0279
Epoch [11/100], Loss: 0.0710
Epoch [12/100], Loss: 0.0276
Epoch [13/100], Loss: 0.0348
Epoch [14/100], Loss: 0.1303
Epoch [15/100], Loss: 0.0450
Epoch [16/100], Loss: 0.0535
Epoch [17/100], Loss: 0.0760
Epoch [18/100], Loss: 0.0284
Epoch [19/100], Loss: 0.0451
Epoch [20/100], Loss: 0.0439
Epoch [21/100], Loss: 0.0344
Epoch [22/100], Loss: 0.0400
Epoch [23/100], Loss: 0.0811
Epoch [24/100], Loss: 0.0500
Epoch [25/100], Loss: 0.1239
Epoch [26/100], Loss: 0.0752
Epoch [27/100], Loss: 0.0870
Epoch [28/100], Loss: 0.0908
Epoch [29/100], Loss: 0.0465
Epoch [30/100], Loss: 0.0365
Epoch [31/100], Loss: 0.0605
Epoch [32/100], Loss: 0.1552
Epoch [33/100], Loss: 0.0204
Epoch [34/100], Loss: 0.0042
Epoch [35/100], Loss: 0

In [0]:
print(di)